In [1]:
# import library heapq
import heapq
# import library time
import time

In [2]:
# mendeklarasikan kelas "pQueue" (priority queue)
# kelas ini menyipan kota-kota yang akan dihitung jaraknya
class pQueue:
    # konstruktor kelas "pQueue" (priority queue)  
    def __init__(self):
        # atribut kota
        self.Kota = []

    # fungsi untuk menambahkan kota ke dalam priority queue 
    def push(self, kota, cost):
        # memasukkan kota ke dalam priority queue
        heapq.heappush(self.Kota, (cost, kota))
    # fungsi untuk mengeluarkan kota dari priority queue
    def pop(self):
        # mengeluarkan kota dari priority queue
        return heapq.heappop(self.Kota)[1]
    # fungsi untuk mengecek apakah priority queue kosong
    def isEmpty(self):
        # mengecek apakah priority queue kosong
        if (self.Kota == []):
            # jika kosong mengembalikan nilai True
            return True
        # jika tidak kosong
        else:
            # mengembalikan nilai False
            return False
    # fungsi untuk mengecek isi dari priority queue
    def check(self):
        # mengecek isi dari priority queue
        print(self.Kota)

In [3]:
# mendeklarasikan kelas "node" 
class node:
    # konstruktor kelas "node"
    def __init__(self, kota, jarak):
        # atribut kota
        self.kota = str(kota)
        # atribut jarak
        self.jarak = str(jarak)

In [4]:
# mendeklarasikan variabel dictionary "actual_cost" untuk menyimpan actual cost antara setiap kota di peta Jawa Timur
actual_cost = {}

# mendeklarasikan fungsi "actualCost" 
def actualCost():
    # membaca file actual_cost.txt yang berisi actual cost antar kota di Peta Jawa Timur
    file = open("actual_cost.txt", 'r')
    # membaca isi file actual_cost.txt
    for string in file:
        # memisahkan isi file actual_cost.txt berdasarkan koma
        line = string.split(',')
        # kota1 = kota asal
        kota1 = line[0]
        # kota2 = kota tujuan
        kota2 = line[1]
        # jarak = jarak antara kota asal dan kota tujuan
        jarak = int(line[2])
        # memasukkan kota tujuan dan jaraknya ke dalam dictionary "actual_cost"
        actual_cost.setdefault(kota1, []).append(node(kota2, jarak)) 
        # memasukkan kota asal dan jaraknya ke dalam dictionary "actual_cost"
        actual_cost.setdefault(kota2, []).append(node(kota1, jarak)) 

In [5]:
# mendeklarasikan fungsi "nilaiHeuristik"
def nilaiHeuristik():
    # mendeklarasikan variabel dictionary "heur" untuk menyimpan jarak (nilai heuristik) suatu kota ke Surabaya di peta Jawa Timur
    heur = {}
    # membaca file nilai_heuristik.txt yang berisi jarak (nilai heuristik) suatu kota ke Surabaya di peta Jawa Timur
    with open("nilai_heuristik.txt", 'r') as file:
        # membaca isi file nilai_heuristik.txt 
        for line in file:
            # memisahkan isi file nilai_heuristik.txt berdasarkan koma
            line = line.strip().split(",")
            # node = nama kota
            node = line[0].strip() 
            # sld = nilai heuristik
            sld = int(line[1].strip()) 
            # memasukkan nama kota dan nilai heuristiknya ke dalam dictionary "heur"
            heur[node] = sld 
    # mengembalikan nilai heuristik
    return heur

# mendeklarasikan fungsi "heuristik" untuk menghitung nilai heuristik dari suatu kota
def heuristik(node, nilai): 
    # mengembalikan nilai heuristik dari suatu kota
    return nilai[node]

In [6]:
# mendeklarasikan fungsi "A_Star" untuk menjalankan algoritma A* search pada peta Jawa Timur 
def A_Star(initial, goal):
    # path = dictionary untuk menyimpan jalur dari kota awal ke kota tujuan
    path = {}
    # jarak = dictionary untuk menyimpan jarak dari kota awal ke kota tujuan
    jarak = {}
    # q = priority queue untuk menyimpan kota-kota yang akan dihitung jaraknya
    q = pQueue()
    # h = nilai heuristik dari suatu kota ke kota tujuan 
    h = nilaiHeuristik()

    # memasukkan kota awal ke dalam priority queue
    q.push(initial, 0)
    # mengisi nilai jarak dari kota awal ke kota awal dengan 0
    jarak[initial] = 0
    # mengisi nilai path dari kota awal ke kota awal dengan None
    path[initial] = None
    # mendeklarasikan variabel "expandedList" untuk menyimpan kota-kota yang sudah dihitung jaraknya
    expandedList = []

    # melakukan perulangan selama priority queue tidak kosong
    while (q.isEmpty() == False):
        # mengeluarkan kota dari priority queue
        current = q.pop()
        # memasukkan kota yang sudah dihitung jaraknya ke dalam variabel "expandedList"
        expandedList.append(current)

        # jika kota yang dihitung jaraknya adalah kota tujuan
        if (current == goal):
            # break perulangan
            break
        # jika kota yang dihitung jaraknya bukan kota tujuan
        for new in actual_cost[current]:
            # menghitung nilai g_cost dari kota yang dihitung jaraknya ke kota tujuan
            g_cost = jarak[current] + int(new.jarak)

            # jika kota yang dihitung jaraknya belum pernah dihitung jaraknya atau nilai g_cost lebih kecil dari nilai jarak kota yang dihitung jaraknya
            if (new.kota not in jarak or g_cost < jarak[new.kota]):
                # mengisi nilai jarak dari kota yang dihitung jaraknya ke kota tujuan dengan nilai g_cost
                jarak[new.kota] = g_cost
                # menghitung nilai f_cost dari kota yang dihitung jaraknya ke kota tujuan
                f_cost = g_cost + heuristik(new.kota, h)
                # memasukkan kota yang dihitung jaraknya ke dalam priority queue
                q.push(new.kota, f_cost)
                # mengisi nilai path dari kota yang dihitung jaraknya ke kota tujuan dengan kota yang dihitung jaraknya
                path[new.kota] = current
    # mengembalikan nilai path, jarak, dan expandedList
    printoutput(initial, goal, path, jarak, expandedList)

In [7]:
# mendeklarasikan fungsi "printoutput" untuk mencetak hasil pencarian rute terpendek dari fungsi "A_Star"
def printoutput(initial, goal, path, jarak, expandedlist):
    # mendeklarasikan variabel "finalpath" untuk menyimpan jalur dari kota awal ke kota tujuan
    finalpath = []
    # mendeklarasikan variabel "i" untuk menyimpan kota tujuan
    i = goal

    # mencetak kota-kota yang sudah dihitung jaraknya
    while (path.get(i) != None):
        # memasukkan kota yang sudah dihitung jaraknya ke dalam variabel "finalpath"
        finalpath.append(i)
        # mengisi variabel "i" dengan kota yang sudah dihitung jaraknya 
        i = path[i]
    # memasukkan kota awal ke dalam variabel "finalpath"
    finalpath.append(initial)
    # membalik urutan isi variabel "finalpath"
    finalpath.reverse()
    print("\t\t")
    print("A* SEARCH")
    print("Asal - Tujuan       : Magetan -> Surabaya")
    print("Kota yang dilewati  : " + str(finalpath))
    print("Jarak               : " + str(jarak[goal]))
    print("\t\t")

In [8]:
# mendeklarasikan fungsi "main" untuk menjalakan program algoritam A* Search
def main():
    # mendeklarasikan variabel "asal" untuk menyimpan kota awal
    asal = "Magetan"
    # mendeklarasikan variabel "tujuan" untuk menyimpan kota tujuan
    tujuan = "Surabaya"
    # memanggil fungsi "actualCost"
    actualCost()
    # mendeklarasikan variabel "startTime" untuk menyimpan waktu awal
    startTime = time.time()
    # memanggil fungsi A_Star
    A_Star(asal, tujuan)
    # mendeklarasikan variabel "endTime" untuk menyimpan waktu akhir
    endTime = time.time()
    # mencetak waktu yang dibutuhkan untuk menjalankan algoritma A* Search
    print("Time: ", endTime - startTime)

# menjalankan fungsi "main"
if __name__ == "__main__":
    # memanggil fungsi "main"
    main()

		
A* SEARCH
Asal - Tujuan       : Magetan -> Surabaya
Kota yang dilewati  : ['Magetan', 'Ngawi', 'Bojonegoro', 'Lamongan', 'Gresik', 'Surabaya']
Jarak               : 144
		
Time:  0.0015609264373779297
